## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nanortalik,GL,2021-11-01 03:27:28,60.1432,-45.2371,33.62,67,70,9.51,broken clouds
1,1,Ostrovnoy,RU,2021-11-01 03:29:20,68.0531,39.5131,30.16,88,86,14.92,overcast clouds
2,2,Qaqortoq,GL,2021-11-01 03:29:07,60.7167,-46.0333,27.10,63,26,6.89,scattered clouds
3,3,Balaka,MW,2021-11-01 03:29:20,-14.9793,34.9558,74.59,59,6,0.69,clear sky
4,4,Mattru,SL,2021-11-01 03:29:20,7.6244,-11.8332,71.47,98,100,0.81,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 87


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Balaka,MW,2021-11-01 03:29:20,-14.9793,34.9558,74.59,59,6,0.69,clear sky
4,4,Mattru,SL,2021-11-01 03:29:20,7.6244,-11.8332,71.47,98,100,0.81,light rain
8,8,New Norfolk,AU,2021-11-01 03:28:53,-42.7826,147.0587,66.65,43,100,3.00,overcast clouds
12,12,Tautira,PF,2021-11-01 03:29:23,-17.7333,-149.1500,84.22,76,59,15.17,broken clouds
13,13,Busselton,AU,2021-11-01 03:29:23,-33.6500,115.3333,62.04,56,73,20.83,broken clouds
14,14,Hasaki,JP,2021-11-01 03:27:28,35.7333,140.8333,68.29,69,100,4.81,overcast clouds
15,15,Ozieri,IT,2021-11-01 03:29:23,40.5860,9.0012,64.02,97,99,4.21,overcast clouds
17,17,Butaritari,KI,2021-11-01 03:24:45,3.0707,172.7902,82.58,73,2,10.89,light rain
19,19,Puerto Ayora,EC,2021-11-01 03:29:24,-0.7393,-90.3518,69.08,91,41,6.60,scattered clouds
21,21,Vaini,TO,2021-11-01 03:29:25,-21.2000,-175.2000,80.76,83,20,11.50,few clouds


In [16]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().sum().count() > 0:
    # 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
    clean_df = preferred_cities_df.dropna()
print(clean_df.count())    

City_ID                342
City                   342
Country                342
Date                   342
Lat                    342
Lng                    342
Max Temp               342
Humidity               342
Cloudiness             342
Wind Speed             342
Current Description    342
dtype: int64


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Balaka,MW,74.59,clear sky,-14.9793,34.9558,
4,Mattru,SL,71.47,light rain,7.6244,-11.8332,
8,New Norfolk,AU,66.65,overcast clouds,-42.7826,147.0587,
12,Tautira,PF,84.22,broken clouds,-17.7333,-149.1500,
13,Busselton,AU,62.04,broken clouds,-33.6500,115.3333,
14,Hasaki,JP,68.29,overcast clouds,35.7333,140.8333,
15,Ozieri,IT,64.02,overcast clouds,40.5860,9.0012,
17,Butaritari,KI,82.58,light rain,3.0707,172.7902,
19,Puerto Ayora,EC,69.08,scattered clouds,-0.7393,-90.3518,
21,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = None
print("Search complete")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Search complete


In [42]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Balaka,MW,74.59,clear sky,-14.9793,34.9558,Sun Village Lodge
4,Mattru,SL,71.47,light rain,7.6244,-11.8332,"Mondkor Town, Bagbo Chiefdom, Bo District"
8,New Norfolk,AU,66.65,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
12,Tautira,PF,84.22,broken clouds,-17.7333,-149.1500,Pension Oaoa
13,Busselton,AU,62.04,broken clouds,-33.6500,115.3333,Observatory Guest House


In [43]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 0.0), zoom_level=1.6)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))